# **EDA for activity variable**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
# feature_engineering
import re

# ploting
import matplotlib.pyplot as plt
# model initial:
import statsmodels.api as sm

#sklearn
from sklearn.preprocessing import OneHotEncoder

In [2]:
#Set import folder
input_folder = "kaggle/input/linking-writing-processes-to-writing-quality/"

In [3]:
# Load train logs
train_logs = pd.read_csv(input_folder + "train_logs.csv",delimiter = ",",header = 0)
# Set the id and event id as index:
train_logs_indexed = train_logs.set_index(['id', 'event_id'])
# Load train scores
train_scores = pd.read_csv(input_folder +"train_scores.csv", delimiter = ",", header = 0)
scores = pd.Series(data = train_scores['score'].values, index = train_scores['id'].values, name = 'score')

In [4]:
train_logs.head(6)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1
5,001519c8,6,107296,107400,104,Input,q,q,q,3,1


In [5]:
train_logs.tail(3)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
8405895,fff05981,3617,2064657,2064765,108,Replace,q,q,q => q,1031,240
8405896,fff05981,3618,2069186,2069259,73,Nonproduction,Leftclick,Leftclick,NoChange,1028,240
8405897,fff05981,3619,2070065,2070133,68,Input,.,.,.,1029,240


In [6]:
train_logs.shape

(8405898, 11)

In [7]:
train_logs.activity.describe()

count     8405898
unique         50
top         Input
freq      6726796
Name: activity, dtype: object

In [8]:
score_60 = scores[scores == 6.0].index.values
score_05 = scores[scores == 0.5].index.values


In [9]:
result = train_logs.groupby('id')['activity'].aggregate([('COUNT', lambda x: len(x))])
result.describe()
# Minimum is 262, max is 12875 steps

,COUNT
count,2471.000000
mean,3401.820316
std,1578.850387
min,262.000000
25%,2193.500000
50%,3082.000000
75%,4301.000000
max,12876.000000


In [10]:
train_logs.activity.unique()

array(['Nonproduction', 'Input', 'Remove/Cut', 'Replace',
       'Move From [284, 292] To [282, 290]',
       'Move From [287, 289] To [285, 287]',
       'Move From [460, 461] To [465, 466]', 'Paste',
       'Move From [905, 1314] To [907, 1316]',
       'Move From [565, 743] To [669, 847]',
       'Move From [669, 847] To [565, 743]',
       'Move From [1041, 1121] To [1496, 1576]',
       'Move From [1455, 1557] To [1323, 1425]',
       'Move From [2268, 2275] To [2247, 2254]',
       'Move From [213, 302] To [902, 991]',
       'Move From [0, 158] To [234, 392]',
       'Move From [460, 465] To [925, 930]',
       'Move From [810, 906] To [816, 912]',
       'Move From [186, 187] To [184, 185]',
       'Move From [140, 272] To [299, 431]',
       'Move From [114, 140] To [272, 298]',
       'Move From [1386, 1450] To [1445, 1509]',
       'Move From [442, 524] To [296, 378]',
       'Move From [408, 414] To [390, 396]',
       'Move From [1144, 1147] To [1142, 1145]',
       'Move 

In [11]:
train_logs.activity.unique().shape

(50,)

In [12]:
t05 = scores[scores == 0.5].index.values
for ids in t05:
    k = train_logs_indexed.loc[ids]['activity'].unique()
    print(k)

['Nonproduction' 'Input' 'Remove/Cut']
['Nonproduction' 'Input' 'Remove/Cut']
['Nonproduction' 'Input' 'Remove/Cut']
['Nonproduction' 'Input' 'Remove/Cut']
['Nonproduction' 'Input' 'Remove/Cut']


In [13]:
t60 = scores[scores == 6].index.values
for ids in t60:
    k = train_logs_indexed.loc[ids]['activity'].unique()
    print(k)

['Nonproduction' 'Input' 'Remove/Cut' 'Replace']
['Nonproduction' 'Input' 'Remove/Cut' 'Replace' 'Paste']
['Nonproduction' 'Input' 'Remove/Cut']
['Nonproduction' 'Input' 'Remove/Cut']
['Nonproduction' 'Input' 'Remove/Cut']
['Nonproduction' 'Input' 'Remove/Cut' 'Replace']
['Nonproduction' 'Input' 'Remove/Cut' 'Replace']
['Nonproduction' 'Input' 'Remove/Cut' 'Replace']
['Nonproduction' 'Input' 'Remove/Cut' 'Paste' 'Replace']
['Nonproduction' 'Input' 'Remove/Cut' 'Replace']
['Nonproduction' 'Input' 'Remove/Cut' 'Replace']
['Nonproduction' 'Input' 'Remove/Cut' 'Replace']
['Nonproduction' 'Input' 'Remove/Cut' 'Replace']
['Nonproduction' 'Input' 'Remove/Cut']
['Nonproduction' 'Input' 'Remove/Cut']
['Nonproduction' 'Input' 'Remove/Cut' 'Replace']
['Nonproduction' 'Input' 'Remove/Cut']
['Nonproduction' 'Input' 'Remove/Cut' 'Replace']
['Nonproduction' 'Paste' 'Input' 'Remove/Cut' 'Replace']
['Nonproduction' 'Input' 'Remove/Cut' 'Replace']
['Nonproduction' 'Input' 'Remove/Cut' 'Replace']
['Nonpr

In [14]:
# Eda for activity 

# Transform all with move into a Move:
# apply function
train_logs.activity = train_logs.activity.apply(lambda x: "Move" if ("Move" in x) else x)
# One hot encode the data:
act_hot = OneHotEncoder(handle_unknown = 'ignore', categories = 'auto', sparse_output = False)
a1 = act_hot.fit_transform(train_logs.activity.values.reshape(-1,1))
a2 = pd.DataFrame(data = a1, columns = ['ac_Input', 'ac_Move', 'ac_NonPro', 'ac_Paste', 'ac_RemCut', 'ac_Replace'])
a2['id'] = train_logs.id.copy()
act = a2.groupby(by = "id").sum()
act.head()

,ac_Input,ac_Move,ac_NonPro,ac_Paste,ac_RemCut,ac_Replace
id,,,,,,
001519c8,2010.0,3.0,120.0,0.0,417.0,7.0
0022f953,1938.0,0.0,254.0,1.0,260.0,1.0
0042269b,3515.0,0.0,175.0,0.0,439.0,7.0
0059420b,1304.0,0.0,99.0,1.0,151.0,1.0
0075873a,1942.0,0.0,72.0,0.0,517.0,0.0


In [102]:
#eda 2 distribution check for activity variable:
# Get the distribution for each kind of score and perform chi-square to detect the score

# act distribution:
act_dist = pd.merge(act, scores, left_index = True, right_index = True)
act_dist = act_dist.groupby('score').sum()
act_dist = act_dist + 1
row_total = np.sum(act_dist, axis = 1)
act_dist = act_dist.apply(lambda x: x / row_total)
# Perform 1 way chi-square test (goodness of fit):
# use at_init since it is the distribution
# xsum = at_init.apply(func = np.sum, axis = 1)
    
# Use chi-square to select the score of the given participant id 
# the below, score values are calculated for efficiency.
score_values = np.arange(start = 0.5, stop = 6.5, step = 0.5)
def compute_score_by_chisquare(fo:pd.Series, distribution):
    # print(score_values)
    total = np.sum(fo)
    # print(total)
    expected_arrays = distribution * total
    # print(expected_arrays)
    chi_stat = []
    for j in range(expected_arrays.shape[0]):
        results = st.chisquare(f_obs = fo, f_exp = expected_arrays.iloc[j])
        chi_stat.append(results[1])

    chi_stat = np.array(chi_stat)
    #print(chi_stat.shape[0])
    # get the maximum chi-square
    idx_1 = np.where(chi_stat == np.partition(chi_stat,-1)[-1])[0][0]
    
    #print(score_index)
    return score_values[idx_1] 


#transform:
act2 = act.apply(lambda x: compute_score_by_chisquare(x, act_dist), axis = 1)
act2.name = "act_chisq"

In [103]:
act2

id
001519c8    2.0
0022f953    2.5
0042269b    6.0
0059420b    2.0
0075873a    1.0
           ... 
ffb8c745    1.0
ffbef7e5    0.5
ffccd6fd    0.5
ffec5b38    6.0
fff05981    2.5
Name: act_chisq, Length: 2471, dtype: float64